# Deploy open-source Large Language Models on Amazon SageMaker

In this notebook, we will show you how to deploy the open-source LLMs from HuggingFace on Amazon SageMaker. The notebook contains five sections:
- Section 1: Deploy Text-to-Text LLM on SageMaker
- Secton 2: Deploy image-to-text LLM on SageMaker (Multimodality)
- Section 3: Deploy embedding model and implement RAG solution using LangChain on SageMaker
- Section 4: (Optional) Run SageMaker Inference Recommender job to determine the cost and performance of the LLM
- Section 5: (Optional) Deploy speech-to-text LLM on SageMaker (note that you can also use Amazon Trascribe to performe the same function)

***
This notebooks is designed to run on `Python 3 Data Science 3.0` kernel in Amazon SageMaker Studio
***

#### 1. Setup development environment

We are going to use the `sagemaker` python SDK to deploy the open source LLMs to Amazon SageMaker. We need to make sure to have an AWS account configured and the `sagemaker` python SDK installed. Please feel free to ignore the error or warning messages in the below cell.

In [ ]:
!pip install "sagemaker==2.180.0" boto3 huggingface_hub --upgrade --quiet
!pip install ipywidgets==7.5.1 langchain==0.0.148 faiss-cpu unstructured --quiet --use-deprecated=legacy-resolver

## Secton 1: (Text-to-Text) Deploy Falcon model to Amazon SageMaker
In this section, we will deploy the open-source [Falcon 7b instruct model](https://huggingface.co/tiiuae/falcon-7b-instruct) on SageMaker for real-time inference. 
To deploy [Falcon-7B-Instruct](https://huggingface.co/tiiuae/falcon-7b-instruct) to Amazon SageMaker we create a `HuggingFaceModel` model class and define our endpoint configuration including the `hf_model_id`, `instance_type` etc. We will use a `g5.2xlarge` instance type.


This is an example on how to deploy the open-source LLMs, like [BLOOM](bigscience/bloom) to Amazon SageMaker for inference using the new Hugging Face LLM Inference Container. We will deploy the 40B-Instruct [Falcon](https://huggingface.co/tiiuae/falcon-40b-instruct) an open-source Chat LLM trained by TII.

The example covers:
1. [Setup development environment](#1-setup-development-environment)
2. [Retrieve the new Hugging Face LLM DLC](#2-retrieve-the-new-hugging-face-llm-dlc)
3. [Deploy Falcon to Amazon SageMaker](#3-deploy-open-assistant-12b-to-amazon-sagemaker)
4. [Run inference and chat with our model](#4-run-inference-and-chat-with-our-model)

## What is Hugging Face LLM Inference DLC?

Hugging Face LLM DLC is a new purpose-built Inference Container to easily deploy LLMs in a secure and managed environment. The DLC is powered by [Text Generation Inference (TGI)](https://github.com/huggingface/text-generation-inference), an open-source, purpose-built solution for deploying and serving Large Language Models (LLMs). TGI enables high-performance text generation using Tensor Parallelism and dynamic batching for the most popular open-source LLMs, including StarCoder, BLOOM, GPT-NeoX, Llama, and T5. 
Text Generation Inference is already used by customers such as IBM, Grammarly, and the Open-Assistant initiative implements optimization for all supported model architectures, including:
* Tensor Parallelism and custom cuda kernels
* Optimized transformers code for inference using [flash-attention](https://github.com/HazyResearch/flash-attention) on the most popular architectures
* Quantization with [bitsandbytes](https://github.com/TimDettmers/bitsandbytes)
* [Continuous batching of incoming requests](https://github.com/huggingface/text-generation-inference/tree/main/router) for increased total throughput
* Accelerated weight loading (start-up time) with [safetensors](https://github.com/huggingface/safetensors)
* Logits warpers (temperature scaling, topk, repetition penalty ...)
* Watermarking with [A Watermark for Large Language Models](https://arxiv.org/abs/2301.10226)
* Stop sequences, Log probabilities
* Token streaming using Server-Sent Events (SSE)

Officially supported model architectures are currently: 
* [BLOOM](https://huggingface.co/bigscience/bloom) / [BLOOMZ](https://huggingface.co/bigscience/bloomz)
* [MT0-XXL](https://huggingface.co/bigscience/mt0-xxl)
* [Galactica](https://huggingface.co/facebook/galactica-120b)
* [SantaCoder](https://huggingface.co/bigcode/santacoder)
* [GPT-Neox 20B](https://huggingface.co/EleutherAI/gpt-neox-20b) (joi, pythia, lotus, rosey, chip, RedPajama, open assistant)
* [FLAN-T5-XXL](https://huggingface.co/google/flan-t5-xxl) (T5-11B)
* [Llama](https://github.com/facebookresearch/llama) (vicuna, alpaca, koala)
* [Starcoder](https://huggingface.co/bigcode/starcoder) / [SantaCoder](https://huggingface.co/bigcode/santacoder)
* [Falcon 7B](https://huggingface.co/tiiuae/falcon-7b) / [Falcon 40B](https://huggingface.co/tiiuae/falcon-40b)

With the new Hugging Face LLM Inference DLCs on Amazon SageMaker, AWS customers can benefit from the same technologies that power highly concurrent, low latency LLM experiences like [HuggingChat](https://hf.co/chat), [OpenAssistant](https://open-assistant.io/), and Inference API for LLM models on the Hugging Face Hub. 

Lets get started!


In [ ]:
import sagemaker
from sagemaker.utils import name_from_base
import boto3
import json
import time
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

bucket = sess.default_bucket()  
aws_region = boto3.Session().region_name
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

#### Retrieve the new Hugging Face LLM DLC

Compared to deploying regular Hugging Face models we first need to retrieve the container uri and provide it to our `HuggingFaceModel` model class with a `image_uri` pointing to the image. To retrieve the new Hugging Face LLM DLC in Amazon SageMaker, we can use the `get_huggingface_llm_image_uri` method provided by the `sagemaker` SDK. This method allows us to retrieve the URI for the desired Hugging Face LLM DLC based on the specified `backend`, `session`, `region`, and `version`. You can find the available versions [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-text-generation-inference-containers)

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="0.9.3"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel

# Define Model and Endpoint configuration parameter
hf_model_id = "tiiuae/falcon-7b-instruct" # model id from huggingface.co/models
instance_type = "ml.g5.2xlarge" # instance type to use for deployment
number_of_gpu = 1 # number of gpus to use for inference and tensor parallelism
health_check_timeout = 600 # Increase the timeout for the health check to 5 minutes for downloading the model
falcon_model_name = name_from_base(hf_model_id.split("/")[-1].replace(".", "-"))
# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
    role=role,
    name=falcon_model_name,
    image_uri=llm_image,
    env={
        'HF_MODEL_ID': hf_model_id,
        # 'HF_MODEL_QUANTIZE': "bitsandbytes", # comment in to quantize
        'SM_NUM_GPUS': json.dumps(number_of_gpu),
        'MAX_INPUT_LENGTH': json.dumps(1900),  # Max length of input text
        'MAX_TOTAL_TOKENS': json.dumps(2048),  # Max length of the generation (including input text)
    }  
)
falcon_endpoint_name = falcon_model_name
falcon_endpoint_name

After we have created the `HuggingFaceModel` we can deploy it to Amazon SageMaker using the `deploy` method. We will deploy the model with the `ml.g5.2xlarge` instance type. 

In [ ]:
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  endpoint_name=falcon_endpoint_name,
  wait=False
)

SageMaker will now create our endpoint and deploy the model to it. This can takes a 10-15 minutes. 

## Run inference and chat with our model

After our endpoint is deployed we can run inference on it. We will use the `predict` method from the `predictor` to run inference on our endpoint. We can inference with different parameters to impact the generation. Parameters can be defined as in the `parameters` attribute of the payload. As of today the TGI supports the following parameters:
* `temperature`: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* `max_new_tokens`: The maximum number of tokens to generate. Default value is 20, max value is 512.
* `repetition_penalty`: Controls the likelihood of repetition, defaults to `null`.
* `seed`: The seed to use for random generation, default is `null`.
* `stop`: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* `top_k`: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is `null`, which disables top-k-filtering.
* `top_p`: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to `null`
* `do_sample`: Whether or not to use sampling ; use greedy decoding otherwise. Default value is `false`.
* `best_of`: Generate best_of sequences and return the one if the highest token logprobs, default to `null`.
* `details`: Whether or not to return details about the generation. Default value is `false`.
* `return_full_text`: Whether or not to return the full text or only the generated part. Default value is `false`.
* `truncate`: Whether or not to truncate the input to the maximum length of the model. Default value is `true`.
* `typical_p`: The typical probability of a token. Default value is `null`.
* `watermark`: The watermark to use for the generation. Default value is `false`.

You can find the open api specification of the TGI in the [swagger documentation](https://huggingface.github.io/text-generation-inference/). 

**Please finish the model deployment for the text-to-text Falcon model. You can move on to the next section to deploy the image-to-text model.**

To see more test cases, you can find an example notebook [here at the SageMaker examples gitrepo.](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart-foundation-models/text-generation-falcon.ipynb)

## Secton 2: (image-to-text) Deploy Blip2 on SageMaker - Multimodality
In this section, we will show you how to deploy the [blip2 model](https://huggingface.co/Salesforce/blip2-opt-6.7b) on SageMaker.

#### Setup
We get DLC image URL for djl-deepspeed 0.21.0 and set SageMaker settings

In [ ]:
from sagemaker import image_uris
import time
import jinja2

session = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = session._region_name
bucket = session.default_bucket()  # bucket to house artifacts
s3_code_prefix = "blip2"  # folder within bucket where code artifact will go
s3_model_prefix = "model_blip2"  # folder within bucket where code artifact will go

img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.22.1")
instance_type = "ml.g5.xlarge"
s3_location = f"s3://{bucket}/djl-serving/"

jinja_env = jinja2.Environment()

# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

Prepare the JSON file to specify the blip2 model used in this example

In [ ]:
# you can choose to deploy any of the models from the list below. In this example, we will deploy blip2-flan-t5-xl

CAPTION_MODELS = {
    'blip-base': 'Salesforce/blip-image-captioning-base',   # 990MB
    'blip-large': 'Salesforce/blip-image-captioning-large', # 1.9GB
    'blip2-2.7b': 'Salesforce/blip2-opt-2.7b',              # 15.5GB
    'blip2-flan-t5-xl': 'Salesforce/blip2-flan-t5-xl',      # 15.77GB
}

blip_model_version = "blip2-flan-t5-xl"
model_names = {
    "caption_model_name": blip_model_version, #@param ["blip-base", "blip-large", "blip2-flan-t5-xl"]
}
with open("blip2/model_name.json",'w') as file:
    json.dump(model_names, file)

In this notebook, we will provide two ways to load the model when deploying to an endpoint.
- Directly load model from Hugging Face 
- Store the model artifacts on S3 and load the model directly from S3

The [Large Model Inference (LMI)](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html) container uses [s5cmd](https://github.com/peak/s5cmd) to download data from S3 which significantly reduces the speed when loading model during deployment. Therefore, we recommend to load the model from S3 by following the below section to download the model from Hugging Face and upload the model on S3. Note that, if you choose to load the model directly from Hugging Face during model deployment, you can skip the below section and jump to the section to prepare the model tarbal file and upload to S3.

### Download the model from Hugging Face and upload the model artifacts on Amazon S3
If you intend to download your copy of the model and upload it to a s3 location in your AWS account, please follow the below steps, else you can skip to the model barball file preparation section.

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path("./blip2-model")
local_model_path.mkdir(exist_ok=True)
model_id = CAPTION_MODELS[blip_model_version]
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_id,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

Upload the model artifacts to S3. If you choose a large Blip2 model, the below cell may take a few minutes to uplaod the files.

In [ ]:
model_artifact = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.s3url={model_artifact}")

In [ ]:
!rm -rf {local_model_path}

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or post-processing of the model's predictions.

However, in this notebook, we demonstrate how to deploy a model with custom inference code.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties`, `model.py`, and `requirements.txt`.
- `serving.properties` is the configuration file that can be used to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration. For more details on the configuration options and an exhaustive list, you can refer the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html).
- `model.py` is the script handles any requests for serving.
- `requirements.txt` is the text file containing any additional pip wheel need to install. 

If you want to download the model from huggingface.co, you can set option.model_id. The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co. If you set the model_id to a s3 url, the DJL will download the model artifacts from s3 and swap the model_id to the actual location of the model artifacts. In your script, you can point to this value to load the pre-trained model.
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which the model needs to be partitioned. This parameter also controls the number of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine, and we are creating 8 partitions then we will have 1 worker per model to serve the requests.


#### Prepare model file.
We can update the configuration for deployment by modifying the [serving.properties](blip2/serving.properties).


In [ ]:
%%writefile blip2/serving.properties
engine = Python
option.tensor_parallel_degree = 1
option.model_id = {{s3url}}

In [ ]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("blip2/serving.properties").open().read())
Path("blip2/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize blip2/serving.properties | cat -n

### Prepare the model tarball file and upload to S3
The code below creates the SageMaker model file (`model.tar.gz`) and upload it to S3. 

In [ ]:
!tar -czvf blip2.tar.gz blip2/

In [ ]:
model_tar_url = sagemaker.s3.S3Uploader.upload("blip2.tar.gz", s3_location)

#### Create SageMaker endpoint
Now we create our SageMaker model. Make sure your execution role has access to your model artifacts and ECR image. Please check out our SageMaker Roles documentation for more details.

In [ ]:
from datetime import datetime

sm_client = boto3.client("sagemaker")

model_name = name_from_base(model_id.split("/")[-1])

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": img_uri, "ModelDataUrl": model_tar_url},
)

In [ ]:
initial_instance_count = 1
variant_name = "AllTraffic"
endpoint_config_name = model_name

production_variants = [
    {
        "VariantName": variant_name,
        "ModelName": model_name,
        "InitialInstanceCount": initial_instance_count,
        "InstanceType": instance_type,
        "ModelDataDownloadTimeoutInSeconds": 1200,
        "ContainerStartupHealthCheckTimeoutInSeconds": 1800
    }
]

endpoint_config = {
    "EndpointConfigName": endpoint_config_name,
    "ProductionVariants": production_variants
}

ep_conf_res = sm_client.create_endpoint_config(**endpoint_config)

In [ ]:
endpoint_name_blip2 = model_name

ep_res = sm_client.create_endpoint(
    EndpointName=endpoint_name_blip2, EndpointConfigName=endpoint_config_name
)

**Once the endpoint creation starts, you can move on to the next section to continue deploying the models to finish remaining task before testing the blip2 model, as this model may take around 15mins to finish deployment**


## Secton 3: (Embedding model) Deploy the GPT-J 6B embedding model on SageMaker and implement RAG solution with LangChain

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor

instance_type = "ml.g5.4xlarge" # instance type to use for deployment
model_version = "*"
env= {
            "SAGEMAKER_MODEL_SERVER_WORKERS": "1", 
            "TS_DEFAULT_WORKERS_PER_MODEL": "1"
    }


In [ ]:
model_id = "huggingface-textembedding-gpt-j-6b-fp16"
# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)
embed_endpoint_name = name_from_base(model_id)

# Retrieve the inference container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=instance_type,
)
model_inference = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=role,
    predictor_cls=Predictor,
    name=model_id,
    env=env,
)
model_predictor_inference = model_inference.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    predictor_cls=Predictor,
    endpoint_name=embed_endpoint_name,
)
print(f"Model {model_id} has been deployed successfully.")

### Wait for all the endpoints are up and running and perform invocations against the endpoints
Now we will wait for all the endpoints to be up and running before testing the models. We will then continue to build the RAG solution with the Falcon and GPT-J embedding model in the next section.

In [ ]:
# wait for the endpoint to be deployed successfully
def wait_for_endpoint(endpoint_name=None):
    describe_endpoint_response = sm_client.describe_endpoint(EndpointName=endpoint_name)

    while describe_endpoint_response["EndpointStatus"] == "Creating":
        describe_endpoint_response = sm_client.describe_endpoint(EndpointName=endpoint_name)
        print(describe_endpoint_response["EndpointStatus"])
        time.sleep(15)

    print(f"endpoint {endpoint_name} is in service now.")
    return


In [ ]:
for ep_name in [falcon_endpoint_name, endpoint_name_blip2, embed_endpoint_name]:
    wait_for_endpoint(ep_name)

#### Evaluate the deployed models

In [ ]:
# Invoke the Falcon model for text generation
chat = llm.predict({
    "inputs": """Hello, how are you?"""
})

print(chat[0]["generated_text"])

In [ ]:
## Invoke the blip 2 model for image-to-text generation
import base64
import json
from PIL import Image
sm_runtime_client = boto3.client("sagemaker-runtime")

image_file = 'test/carcrash-ai.jpeg'
def encode_image(img_file):
    with open(img_file, "rb") as image_file:
        img_str = base64.b64encode(image_file.read())
        base64_string = img_str.decode("latin1")
    return base64_string

# prepare inputs
base64_string = encode_image(image_file)
question = "What happened in the image?"
inputs = {"prompt": f"Question: {question}/n Answer:", "image": base64_string}

# invoke endpoint
response = sm_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name_blip2, Body=json.dumps(inputs)
)

# show results
raw_image = Image.open(image_file).convert('RGB')
display(raw_image)
print(f"This is the quesiton: {question}")
print(f"This is the answer from the blip2 model: {response['Body'].read().decode()}")

In [ ]:
# invoke the embedding model
input_str = {"text_inputs": question}
output = sm_runtime_client.invoke_endpoint(
    EndpointName=embed_endpoint_name, Body=json.dumps(input_str), ContentType="application/json"
)
embeddings = output["Body"].read().decode("utf-8")
print(embeddings)

Now we have tested all the deployed endpoints and verified we can successfully invoke these endpoints. We can continue to build and test the RAG solution using the Falcon and the embedding models.

### Use RAG based approach with [LangChain](https://python.langchain.com/en/latest/index.html) and SageMaker endpoints to build a simplified question and answering application.


We plan to use document embeddings to fetch the most relevant documents in our document knowledge library and combine them with the prompt that we provide to LLM.

To achieve that, we will do following.

1. **Generate embedings for each of document in the knowledge library with SageMaker GPT-J-6B embedding model.**
2. **Identify top K most relevant documents based on user query.**
    - 2.1 **For a query of your interest, generate the embedding of the query using the same embedding model.**
    - 2.2 **Search the indexes of top K most relevant documents in the embedding space using in-memory Faiss search.**
    - 2.3 **Use the indexes to retrieve the corresponded documents.**
3. **Combine the retrieved documents with prompt and question and send them into SageMaker LLM.**



Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt -- maximum sequence length of 1024 tokens. 

---
To build a simiplied QA application with LangChain, we need: 
1. Wrap up our SageMaker endpoints for embedding model and LLM into `langchain.embeddings.SagemakerEndpointEmbeddings` and `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. That requires a small overwritten of `SagemakerEndpointEmbeddings` class to make it compatible with SageMaker embedding mdoel.
2. Prepare the dataset to build the knowledge data base. 

---

Wrap up our SageMaker endpoints for embedding model into `langchain.embeddings.SagemakerEndpointEmbeddings`. That requires a small overwritten of `SagemakerEndpointEmbeddings` class to make it compatible with SageMaker embedding mdoel.

In [ ]:
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from typing import Any, Dict, List, Optional

class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int = 5) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size

        for i in range(0, len(texts), _chunk_size):
            response = self._embedding_func(texts[i : i + _chunk_size])
            print
            results.extend(response)
        return results


class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        embeddings = response_json["embedding"]
        return embeddings


content_handler = ContentHandler()

embeddings = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=embed_endpoint_name,
    region_name=aws_region,
    content_handler=content_handler,
)

Next, we wrap up our SageMaker endpoints for LLM into `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. 

In [ ]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint

parameters = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1
}


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        self.len_prompt = len(prompt)
        input_str = json.dumps({"inputs": prompt, "parameters": {**model_kwargs}})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = output.read()
        res = json.loads(response_json)
        print(res)
        ans = res[0]['generated_text'][self.len_prompt:]
        return ans 


content_handler = ContentHandler()

sm_llm = SagemakerEndpoint(
    endpoint_name=falcon_endpoint_name,
    region_name=aws_region,
    model_kwargs=parameters,
    content_handler=content_handler,
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader

Use langchain to read the `txt` data. There are multiple built-in functions in LangChain to read different format of files such as `csv`, `html`, and `pdf`. For details, see [LangChain document loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html).

In [ ]:
loader = DirectoryLoader("./data/", glob="**/*.txt")
documents = loader.load()

We generate embedings for each of document in the knowledge library with SageMaker GPT-J-6B embedding model.

In [ ]:
docsearch = FAISS.from_documents(documents, embeddings)

In [ ]:
question = "what is the recommended way to first customize a foundation model?"

Based on the question above, we then **identify top K most relevant documents based on user query, where K = 3 in this setup**.

In [ ]:
docs = docsearch.similarity_search_with_score(question)

In [ ]:
source = []
context = []
for doc, score in docs:
    context.append(doc)
    source.append(doc.metadata['source'].split('/')[-1])

Finally, we **combine the retrieved documents with prompt and question and send them into SageMaker LLM.** 

We define a customized prompt as below.

In [ ]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.:\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
chain = load_qa_chain(llm=sm_llm, prompt=PROMPT)

In [ ]:
result = chain({"input_documents": context, "question": question}, return_only_outputs=True)["output_text"]
print(result)

Congratulations on finishing the model deployment !!! Now you can go back to the instructions and test the chatbot UI.

---------------------------------------------------------------------

## Secton 4: (Optional) Understand the model hosting performance using SageMaker Inference Recommender 

[Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html) is a capability of Amazon SageMaker that reduces the time required to get machine learning (ML) models in production by automating load testing and model tuning across SageMaker ML instances. You can use Inference Recommender to deploy your model to a real-time or serverless inference endpoint that delivers the best performance at the lowest cost. Inference Recommender helps you select the best instance type and configuration (such as instance count, container parameters, and model optimizations) or serverless configuration (such as max concurrency and memory size) for your ML models and workloads.

In [ ]:
# !tar -czvf payload.tar.gz test/payload.json

In [ ]:
# s3_location = f"s3://{bucket}/sagemaker/InferenceRecommender/{model_name}"
# payload_tar_url = sagemaker.s3.S3Uploader.upload("payload.tar.gz", s3_location)
# print(payload_tar_url)

In [ ]:
# job_name = f"{falcon_model_name}-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
# response = sm_client.create_inference_recommendations_job(
#     JobName=job_name,
#     JobType='Default',
#     RoleArn=role,
#     InputConfig={
#         'ContainerConfig': {
#             'Domain': 'NATURAL_LANGUAGE_PROCESSING',
#             'Task': 'TEXT_GENERATION',
#             'PayloadConfig': {
#                 'SamplePayloadUrl': payload_tar_url,
#                 'SupportedContentTypes': ["application/json"],
#             },
#             #specify the instance types you would like to test out
#             'SupportedInstanceTypes': ['ml.g5.2xlarge'], 
#             'SupportedEndpointType': 'RealTime'
#         },
#         'ModelName': falcon_model_name
#     },
# )

In [ ]:
# # uncomment this section to wait for the inference job to finish
# describe_IR_job_response = sm_client.describe_inference_recommendations_job(JobName=job_name)

# while describe_IR_job_response["Status"] in ["IN_PROGRESS", "PENDING"]:
#     describe_IR_job_response = sm_client.describe_inference_recommendations_job(JobName=job_name)
#     print(describe_IR_job_response["Status"])
#     time.sleep(15)
    
# print(f'Inference Recommender job {job_name} has finished with status {describe_IR_job_response["Status"]}.')

Now, let's use the inference recommender job results to calculate the approximate invocation cost for the LLM endpoint.

In [ ]:
# describe_IR_job_response = sm_client.describe_inference_recommendations_job(JobName=job_name)
# describe_IR_job_response['InferenceRecommendations']

The inference recomender job reports the below metrics: 
- 'ModelLatency'
- 'CostPerInference'
- 'CostPerHour'
- 'MaxInvocations' per minute

and more.

Note that the sample json input file consists of 6,200 characters, which is around 1550 tokens per invocation (1 token is approximately 4 characters). To calculate the approximate cost per 1K tokens, you can do the inference many times (with average payload size) and get the best token/s you get through the experiment (different instance types can result in different throughput, model latency, and cost). Then we will calculate the per token per second invocation price and multiply by 1,000. You can also use per invocation cost divide by the tokens per invocation and multiply by 1,000. The calculated price should be similar. SageMaker also supports auto-scaling to scale your endpoint out/in to save cost based on the invocation traffic pattern.

In [ ]:
# metrics = describe_IR_job_response['InferenceRecommendations'][0]['Metrics']
# token_per_sec = round(metrics['MaxInvocations']*1550/60, 2)
# cost_per_sec = round(metrics['CostPerHour']/3600, 5)
# cost_per_1k_token = round(cost_per_sec/token_per_sec * 1000, 5)
# print("According to the Inference recommender job, the corresponding metrices are as below: /n")
# print(f"Max tokens per second is about {token_per_sec}")
# print(f"Cost per second is about ${cost_per_sec}")
# print(f"Cost per 1k tokens is about ${cost_per_1k_token}")

## Section 5: (Optional) Deploy speech-to-audio LLM on SageMaker
In this section, we will deploy the open-source [Whisper model](https://huggingface.co/openai/whisper-large-v2) to SageMaker real-time hosting. 

In [ ]:
# import sagemaker
# import boto3
# from sagemaker.huggingface import HuggingFaceModel
# from sagemaker.serializers import DataSerializer
# import sagemaker
# import json
# client = boto3.client('runtime.sagemaker')

# try:
# 	role = sagemaker.get_execution_role()
# except ValueError:
# 	iam = boto3.client('iam')
# 	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# # Hub Model configuration. https://huggingface.co/models
# hub = {
# 	'HF_MODEL_ID':'openai/whisper-large-v2',
# 	'HF_TASK':'automatic-speech-recognition',
# }

# # create Hugging Face Model Class
# huggingface_model = HuggingFaceModel(
# 	transformers_version='4.26.0',
# 	pytorch_version='1.13.1',
# 	py_version='py39',
# 	env=hub,
# 	role=role, 
# )
# endpoint_name="whisper-large-v2"
# # deploy model to SageMaker Inference
# predictor = huggingface_model.deploy(
#     endpoint_name=endpoint_name,
# 	initial_instance_count=1, # number of instances
# 	instance_type='ml.g5.xlarge' # ec2 instance type
# )



In [ ]:
# file = "test/test.webm"
# with open(file, "rb") as f:
# 	data = f.read()

In [ ]:
# # option 1: using SageMaker python SDK
# predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name, 
#                                           serializer=DataSerializer(content_type='audio/x-audio'),
#                                          )
# predictor.predict(data)

In [ ]:
# # option 2: using boto3 invoke_endpoint api
# response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='audio/x-audio', Body=data)
# json.loads(response['Body'].read())

## Section 6: (Optional) Deploy Llama 2 on SageMaker

The Llama 2 foundation models developed by Meta are available for customers through [Amazon SageMaker JumpStart](https://aws.amazon.com/sagemaker/jumpstart/). Llama 2 is an auto-regressive language model that uses an optimized transformer architecture. Llama 2 is intended for commercial and research use in English. It comes in a range of parameter sizes—7 billion, 13 billion, and 70 billion—as well as pre-trained and fine-tuned variations. According to Meta, the tuned versions use supervised fine-tuning (SFT) and reinforcement learning with human feedback (RLHF) to align to human preferences for helpfulness and safety. Llama 2 was pre-trained on 2 trillion tokens of data from publicly available sources. The tuned models are intended for assistant-like chat, whereas pre-trained models can be adapted for a variety of natural language generation tasks.

You can follow the [blog post](https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/) to deploy the Llama 2 from JumpStart and replace the chatbot LLM model name with the Llama 2 endpoint name then test the performance of the model

## Clean up
Uncomment below cell and use the boto3 api calls to delete all the resources created in the workshop. If you no longer need the model artifacts, you should go to the S3 bucket and delete the resources stored on S3. 

In [ ]:
# client.delete_endpoint(EndpointName=falcon_endpoint_name)
# client.delete_endpoint(EndpointName=endpoint_name_blip2)
# client.delete_endpoint(EndpointName=embed_endpoint_name)